In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, IntegerType

In [2]:
spark = SparkSession.builder.appName("Spark Chap5 Parctice").enableHiveSupport().getOrCreate()

#### Spark SQL UDFs

In [8]:
def cubed(s):
    return s * s * s

In [9]:
spark.udf.register("cubed",cubed, LongType())

<function __main__.cubed(s)>

In [10]:
spark.range(1,9).createOrReplaceTempView("udf_test")

In [11]:
spark.sql("SELECT id,cubed(id) as id_cubed FROM udf_test").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
+---+--------+



##### Evaluation order and null checking in Spark SQL

In [21]:
def strlen(s):
    return len(s)

# spark.udf.register("strlen", lambda s: len(s), "int") #Alternate 
spark.udf.register("strlen", strlen, IntegerType())

<function __main__.strlen(s)>

In [30]:
spark.udf.register("strlen_nullsafe", lambda s: len(s) if not s is None else -1, "int")
# spark.sql("select s from test1 where s is not null and strlen_nullsafe(s) > 1") // ok
# spark.sql("select s from test1 where if(s is not null, strlen(s), null) > 1")   // ok

<function __main__.<lambda>(s)>

#### Speeding up and distributing PySpark UDFs with Pandas UDFs

In [32]:
# Pandas udfs over PySpark udfs because, 
# PySpark udfs required data movement btwn JVM and Python,
# which is quite expensive.

# To resolve this Pandas UDFs (also known as vectorized UDFs) were introduced
# Panda UDFs uses Apache Arrow to transfer data and Pandas to work with data.


In [ ]:
# Pandas UDfs were split into two API categories
# 1. Pandas UDFs
# 2. Pandas Function APIs

In [40]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [66]:
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

In [72]:
cubed_udf = pandas_udf(cubed, returnType=LongType())

x = pd.Series([1,2,3])

print(cubed(x))

df = spark.range(1,4)

df.select("id", cubed_udf(col("id"))).show()

0     1
1     8
2    27
dtype: int64


Py4JJavaError: An error occurred while calling o415.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 32.0 failed 1 times, most recent failure: Lost task 1.0 in stage 32.0 (TID 61, 10.0.2.93, executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at io.netty.util.internal.PlatformDependent.directBuffer(PlatformDependent.java:490)
	at io.netty.buffer.NettyArrowBuf.getDirectBuffer(NettyArrowBuf.java:243)
	at io.netty.buffer.NettyArrowBuf.nioBuffer(NettyArrowBuf.java:233)
	at io.netty.buffer.ArrowBuf.nioBuffer(ArrowBuf.java:245)
	at org.apache.arrow.vector.ipc.message.ArrowRecordBatch.computeBodyLength(ArrowRecordBatch.java:222)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.serialize(MessageSerializer.java:240)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeRecordBatch(ArrowWriter.java:132)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeBatch(Arr